In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine 
from sqlalchemy import create_engine, text

PostgreSQL_HOST = '127.0.0.1'
PostgreSQL_PORT = '5432'
PostgreSQL_USER = 'postgres'
PostgreSQL_PASSWORD = '123456'
PostgreSQL_DB = 'sql_advanced'

engine = create_engine('postgresql+psycopg2://%s:%s@%s:%s/%s'
                           % (PostgreSQL_USER, PostgreSQL_PASSWORD, PostgreSQL_HOST, PostgreSQL_PORT, PostgreSQL_DB))

In [2]:
CarSale_df = pd.read_sql_query('SELECT * FROM CarSale', engine) 
CarSale_df

,car_id,sale_dt,sale_count
0,10001,2023-05-01,1
1,10001,2023-05-02,2
2,10001,2023-05-03,3
3,10001,2023-05-04,4
4,10001,2023-06-05,5
5,10001,2023-06-06,6
6,10001,2023-06-07,7
7,10001,2023-06-08,8
8,10002,2023-05-01,10
9,10002,2023-05-02,11


In [3]:
query_1 = """
    select car_id,sale_dt,sale_count, 
        sum(sale_count) over(partition by car_id order by sale_count RANGE BETWEEN CURRENT ROW AND 2 FOLLOWING) as num 
    from CarSale ;
"""
df = pd.read_sql_query(query_1, engine) 
df

,car_id,sale_dt,sale_count,num
0,10001,2023-05-01,1,6
1,10001,2023-05-02,2,9
2,10001,2023-05-03,3,12
3,10001,2023-05-04,4,15
4,10001,2023-06-05,5,18
5,10001,2023-06-06,6,21
6,10001,2023-06-07,7,15
7,10001,2023-06-08,8,8
8,10002,2023-06-05,4,15
9,10002,2023-06-06,5,18


In [4]:
df = pd.read_sql_query("""select * from salarywz""", engine) 
df

,year,person_id,person_name,country_name,salary
0,2021,100,鲁班,战国,10000
1,2022,100,鲁班,战国,10001
2,2023,100,鲁班,战国,9999
3,2022,101,白起,战国,9856
4,2023,101,白起,战国,10003
5,2022,102,蔡文姬,三国,9857
6,2023,102,蔡文姬,三国,10523
7,2020,102,蔡文姬,三国,10006
8,2021,103,孙悟空,唐朝,10007
9,2022,103,孙悟空,唐朝,10568


In [5]:
query_3 = """
select *, 
    MAX(salary) OVER() as max_salay,
    MIN(salary) OVER(partition by country_name) as min_salary_country
from salarywz
"""
df_3 = pd.read_sql_query(query_3, engine) 
df_3

,year,person_id,person_name,country_name,salary,max_salay,min_salary_country
0,2022,102,蔡文姬,三国,9857,10568,9857
1,2020,102,蔡文姬,三国,10006,10568,9857
2,2023,102,蔡文姬,三国,10523,10568,9857
3,2020,103,孙悟空,唐朝,10010,10568,10007
4,2021,103,孙悟空,唐朝,10007,10568,10007
5,2022,103,孙悟空,唐朝,10568,10568,10007
6,2023,103,孙悟空,唐朝,10325,10568,10007
7,2022,100,鲁班,战国,10001,10568,9856
8,2021,100,鲁班,战国,10000,10568,9856
9,2023,101,白起,战国,10003,10568,9856


In [6]:
query_4 = """
SELECT person_name,country_name, MIN(min_salary_country) AS min_salary_country 
FROM ( 
    SELECT person_name,country_name, MIN(salary) OVER (PARTITION BY 	country_name) AS min_salary_country 
    FROM salarywz 
    ) AS subquery 
GROUP BY person_name,country_name;
"""
df_4 = pd.read_sql_query(query_4, engine) 
df_4

,person_name,country_name,min_salary_country
0,蔡文姬,三国,9857
1,孙悟空,唐朝,10007
2,鲁班,战国,9856
3,白起,战国,9856
